# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, Dataset, Model, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

### Kaggle - Housing Prices Competition for Kaggle Learn Users
We will be using the "Housing Prices Competition for Kaggle Learn Users" training and test datasets for this capstone project.

This is a regression competition in which competitors try to predict the price of the houses in the test dataset using the training dataset.

The original dataset was first published by Dean De Cock in his paper [Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project](https://www.researchgate.net/publication/267976209_Ames_Iowa_Alternative_to_the_Boston_Housing_Data_as_an_End_of_Semester_Regression_Project) at Journal of Statistics Education (November 2011).

For competition purposes, approximately all of the data has been divided into two parts: "training dataset" and "test dataset" We will be using the training dataset for training and the test dataset for submission to the competition.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py#provisioning-configuration-vm-size-----vm-priority--dedicated---min-nodes-0--max-nodes-none--idle-seconds-before-scaledown-none--admin-username-none--admin-user-password-none--admin-user-ssh-key-none--vnet-resourcegroup-name-none--vnet-name-none--subnet-name-none--tags-none--description-none--remote-login-port-public-access--notspecified--

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb


# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# Get the training data in the data folder from github
web_path = ['https://github.com/ErkanHatipoglu/nd00333-capstone/tree/master/starter_file/data/train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x,y)

# Concat the data for automl experiment
train_data = pd.concat([x_train, y_train], axis=1)
valid_data = pd.concat([x_test, y_test], axis=1)

In [ ]:
# automl_config requires TabularDataset as a result we need to
# create a dataset from pandas dataframe
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#create-a-filedataset
print(type(train_data))

# create data folder if not exist 
if "data" not in os.listdir():
    os.mkdir("./data")

# convert train dataframe
train_path = 'data/train_cleaned.csv'
train_data.to_csv(train_path)

# convert valid dataframe
valid_path = 'data/valid_cleaned.csv'
valid_data.to_csv(valid_path)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
train_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/train_cleaned.csv'))])

# create a dataset referencing the cloud location
valid_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/valid_cleaned.csv'))])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'mean_absolute_error',
    "max_cores_per_iteration"=-1,
    "max_concurrent_iterations"=4, 
    "n_cross_validations"=5,
    "enable_early_stopping"= True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = ws.compute_targets['cpu-cluster'],
                             task = "regression",
                             training_data=train_dataset,
                             validation_data=valid_dataset,
                             label_column_name="SalePrice",   
                             path = project_folder
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status()=="Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.

# https://github.com/MicrosoftLearning/DP100/blob/master/08B%20-%20Using%20Automated%20Machine%20Learning.ipynb
# Get the best run object
best_run, fitted_model = remote_run.get_output()
print("Summary:")
print(remote_run.summary())
print("********************\n")
print("Best run:")
print(best_run)
print("********************\n")
print("Estimator:")
print(fitted_model.steps[-1])
print("********************\n")
print("Model:")
print(fitted_model)
print("********************\n")
best_run_metrics = best_run.get_metrics()
print('Accuracy:', best_run_metrics['accuracy'])
print("********************\n")

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
#TODO: Save the best model
# https://knowledge.udacity.com/questions/357007
os.makedirs('outputs', exist_ok=True)
joblib.dump(best_run, 'outputs/model.joblib')


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register model
model = Model.register_model(model_path='outputs/model.pkl', model_name='best_automl_run')
# Check model
for model in Model.list(ws):
    print("Model Name: {}\n".format(model.name))
    print(model)
    print("********************\n")

In [ ]:
environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [ ]:
with open('score.py') as f:
    print(f.read())

In [ ]:
service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
# Enable application insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_payload = json.dumps({
    'data': valid_data.tolist(),
    'method': 'predict'
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
# Delete the service
service.delete()

In [ ]:
# Delete compute cluster
cpu_cluster.delete()

# References
- Cock, Dean. (2011). Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project. Journal of Statistics Education. 19. 10.1080/10691898.2011.11889627.
- [Deployment to Cloud Example](https://github.com/ErkanHatipoglu/MachineLearningNotebooks/tree/master/how-to-use-azureml/deployment/deploy-to-cloud)